

## Deep learning: CNN Kernels





In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import Image
import timeit

%matplotlib inline



In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
n_train = x_train.shape[0]
n_test = x_test.shape[0]

We scale the data to be in $[-1,1]$.

In [3]:
x_train, x_test = x_train / 127.5 - 1, x_test / 127.5 - 1

The size of each data points is $28 \times 28$. While this format will be useful when using CNNs, we will vectorize the datapoints for visualization and preliminary questions.

In [4]:
nb_features = np.prod(x_train.shape[1:])
x_train.resize((n_train, nb_features))
x_test.resize((n_test, nb_features))

### 4. Convolutional Neural Network (8 points)
Now that we are more familiar with the MLP algorithm, it is time to see how it compares with a Convolutional Neural Network (CNN). CNNs leverage dependencies between neighbouring pixels, making them more efficient and light weight compared to their fully connected counter part. In this section we will implement a Class CNN similar to the one we defined before for MLP.

In [5]:
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

In [6]:
cnn_acc = []

##### Task 4.1
Complete the function 'create_model' of the class CNN above. Implement a CNN of the shape [32, 64, 128]; where 32, 64, 128 represent the number of convolutional filters for each hidden layer. We will use a kernel size of size $4\times4$. 

Use a stride of 1 in the first convolutional layer, followed by a stride of 2 for the following layers (a stride of two help downsampling without requiring the use of pooling layers). Vectorize the obtained output using tf.layer.flatten, and end the model with a fully connected layer of 10 neurons. Use ReLU as the non linear activation for the hidden layers.

   The function 'create_model' to complete defines the class variables: 

   + self.logits $\in \mathbb{R^{10}}$ containing the output without activation of the last __<font color='red'>fully connected layer</font>.__ 

   + self.preds $\in \mathbb{R^{10}}$ containing posterior probabilities.

- Using self.logits complete the method 'compute_loss' that takes the labels and the predicted logits to return the corresponfing cross-entropy loss. 

Hints: 
+ You may use tf.layers.conv2d to implement a convolutional layer. 
+ To obtain the probabilities you must normalize your outputs in a way that their sum is equal to one using a softmax function.

#### CNN 1: 3 HL filters[32, 64, 128] k(4x4)

In [7]:
class CNN:
    def __init__(self, x_train, y_train, output_dir, lr=0.001, nb_epochs=10, batch_size=50):
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.batch_size = batch_size
        self.nb_epochs = nb_epochs
        self.nb_images, self.edge, _ = x_train.shape
        self.nb_iterations = self.nb_images // batch_size
        self.output_dir = output_dir
        self.im = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.labels = tf.placeholder(tf.float32, [None, 10])
        self.x_train = x_train[:,:,:, np.newaxis]
        self.y_train = y_train
        
    def create_model(self):
        with tf.variable_scope('CNN', reuse=tf.AUTO_REUSE):
           
            hl_1 = tf.layers.conv2d(self.im, 32, kernel_size=(4, 4), strides=1, activation=tf.nn.relu, name='hl_1')
            hl_2 = tf.layers.conv2d(hl_1, 64, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_2')
            hl_3 = tf.layers.conv2d(hl_2, 128, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_3')

            self.logits = tf.layers.dense(tf.layers.flatten(hl_3),10,activation=None, name='logits')
            self.preds = tf.nn.softmax(self.logits, name='preds')
            #########################################
    
    def compute_loss(self):
        with tf.variable_scope('loss'):

            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.labels,logits=self.logits))
            #########################################
            self.loss_summ = tf.summary.scalar("softmax_loss", self.loss)
            
    def optimizer(self):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5)
            self.model_vars = tf.trainable_variables()
            self.trainer = optimizer.minimize(self.loss, var_list=self.model_vars)

CNNs leverage dependencies between neighbouring pixels, however this information is partially lost when we vectorized our data. For training CNNs we will need to recover our initial shape of $N \times 28 \times 28$.

In [8]:
edge = int(np.sqrt(nb_features))
x_train.resize([n_train, edge, edge])
x_test.resize([n_test, edge, edge])

We instantiate our CNN class, start the corresponging tensorflow session and initiate the trainable variables. 

In [9]:
model = CNN(x_train, y_train, './CNN_logdir/', 0.001, 2, 50)
model.create_model()
model.compute_loss()
model.optimizer()
init = (tf.global_variables_initializer(),
        tf.local_variables_initializer())

saver = tf.train.Saver()
summary =tf.Summary()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.summary.FileWriter(model.output_dir)
writer.add_graph(sess.graph)
if not os.path.exists(model.output_dir):
    os.makedirs(model.output_dir)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


We start training:

In [10]:
start_time = timeit.default_timer()

for epoch in range(model.nb_epochs):
    randomize = np.arange(x_train.shape[0])
    np.random.shuffle(randomize)
    x_in = model.x_train[randomize,:]
    y_in = model.y_train[randomize,:]
    for i in range(model.nb_iterations):
        input_x_train = x_in[i*model.batch_size: (i+1)*model.batch_size]
        input_y_train = y_in[i*model.batch_size: (i+1)*model.batch_size]
        _ , preds, loss, loss_summ = sess.run([model.trainer, model.preds, model.loss, model.loss_summ], 
                                 feed_dict={model.im: input_x_train, 
                                            model.labels: input_y_train})
        y_preds = np.argmax(preds, axis=1)
        y_real = np.argmax(input_y_train, axis=1)
        acc_train = np.mean((y_preds==y_real)*1)
        print('Epoch %d, Iteration %d, loss %.3f, batch accuracy %.3f' %(epoch, i, loss, acc_train))
        writer.add_summary(loss_summ, epoch * model.nb_iterations + i)
    saver.save(sess, model.output_dir, global_step=epoch)  
# sess.close()

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Epoch 0, Iteration 0, loss 2.312, batch accuracy 0.040
Epoch 0, Iteration 1, loss 2.233, batch accuracy 0.260
Epoch 0, Iteration 2, loss 2.244, batch accuracy 0.240
Epoch 0, Iteration 3, loss 2.193, batch accuracy 0.140
Epoch 0, Iteration 4, loss 2.115, batch accuracy 0.300
Epoch 0, Iteration 5, loss 1.937, batch accuracy 0.500
Epoch 0, Iteration 6, loss 1.829, batch accuracy 0.400
Epoch 0, Iteration 7, loss 1.605, batch accuracy 0.520
Epoch 0, Iteration 8, loss 1.349, batch accuracy 0.700
Epoch 0, Iteration 9, loss 1.480, batch accuracy 0.420
Epoch 0, Iteration 10, loss 1.000, batch accuracy 0.760
Epoch 0, Iteration 11, loss 1.032, batch accuracy 0.700
Epoch 0, Iteration 12, loss 0.808, batch accuracy 0.760
Epoch 0, Iteration 13, loss 0.860, batch accuracy 0.720
Epoch 0, Iteration 14, loss 0.953, batch accuracy 0.700
Epoch 0, Iteration 15, loss 0.644, batch accuracy 0.820
Epoch 0, Iteration 16, loss 0.615, batch accuracy 0.760
Epoch 0, Iteration 17, loss 0.777, batch accuracy 0.820
Ep

Epoch 0, Iteration 146, loss 0.196, batch accuracy 0.940
Epoch 0, Iteration 147, loss 0.360, batch accuracy 0.920
Epoch 0, Iteration 148, loss 0.239, batch accuracy 0.920
Epoch 0, Iteration 149, loss 0.161, batch accuracy 0.940
Epoch 0, Iteration 150, loss 0.186, batch accuracy 0.940
Epoch 0, Iteration 151, loss 0.120, batch accuracy 0.980
Epoch 0, Iteration 152, loss 0.180, batch accuracy 0.900
Epoch 0, Iteration 153, loss 0.077, batch accuracy 1.000
Epoch 0, Iteration 154, loss 0.128, batch accuracy 0.940
Epoch 0, Iteration 155, loss 0.090, batch accuracy 0.980
Epoch 0, Iteration 156, loss 0.113, batch accuracy 0.940
Epoch 0, Iteration 157, loss 0.153, batch accuracy 0.960
Epoch 0, Iteration 158, loss 0.184, batch accuracy 0.940
Epoch 0, Iteration 159, loss 0.343, batch accuracy 0.900
Epoch 0, Iteration 160, loss 0.165, batch accuracy 0.940
Epoch 0, Iteration 161, loss 0.132, batch accuracy 0.920
Epoch 0, Iteration 162, loss 0.157, batch accuracy 0.940
Epoch 0, Iteration 163, loss 0.

Epoch 0, Iteration 290, loss 0.077, batch accuracy 0.980
Epoch 0, Iteration 291, loss 0.253, batch accuracy 0.920
Epoch 0, Iteration 292, loss 0.145, batch accuracy 0.940
Epoch 0, Iteration 293, loss 0.104, batch accuracy 0.980
Epoch 0, Iteration 294, loss 0.308, batch accuracy 0.960
Epoch 0, Iteration 295, loss 0.074, batch accuracy 0.980
Epoch 0, Iteration 296, loss 0.235, batch accuracy 0.920
Epoch 0, Iteration 297, loss 0.075, batch accuracy 1.000
Epoch 0, Iteration 298, loss 0.079, batch accuracy 0.960
Epoch 0, Iteration 299, loss 0.115, batch accuracy 0.960
Epoch 0, Iteration 300, loss 0.145, batch accuracy 0.980
Epoch 0, Iteration 301, loss 0.083, batch accuracy 0.980
Epoch 0, Iteration 302, loss 0.076, batch accuracy 0.960
Epoch 0, Iteration 303, loss 0.087, batch accuracy 0.960
Epoch 0, Iteration 304, loss 0.153, batch accuracy 0.960
Epoch 0, Iteration 305, loss 0.090, batch accuracy 0.940
Epoch 0, Iteration 306, loss 0.062, batch accuracy 0.980
Epoch 0, Iteration 307, loss 0.

Epoch 0, Iteration 434, loss 0.200, batch accuracy 0.980
Epoch 0, Iteration 435, loss 0.024, batch accuracy 1.000
Epoch 0, Iteration 436, loss 0.059, batch accuracy 0.980
Epoch 0, Iteration 437, loss 0.236, batch accuracy 0.880
Epoch 0, Iteration 438, loss 0.085, batch accuracy 0.960
Epoch 0, Iteration 439, loss 0.116, batch accuracy 0.980
Epoch 0, Iteration 440, loss 0.057, batch accuracy 0.980
Epoch 0, Iteration 441, loss 0.066, batch accuracy 0.980
Epoch 0, Iteration 442, loss 0.039, batch accuracy 1.000
Epoch 0, Iteration 443, loss 0.126, batch accuracy 0.960
Epoch 0, Iteration 444, loss 0.114, batch accuracy 0.960
Epoch 0, Iteration 445, loss 0.054, batch accuracy 0.980
Epoch 0, Iteration 446, loss 0.060, batch accuracy 0.980
Epoch 0, Iteration 447, loss 0.014, batch accuracy 1.000
Epoch 0, Iteration 448, loss 0.075, batch accuracy 0.960
Epoch 0, Iteration 449, loss 0.056, batch accuracy 0.960
Epoch 0, Iteration 450, loss 0.049, batch accuracy 1.000
Epoch 0, Iteration 451, loss 0.

Epoch 0, Iteration 578, loss 0.037, batch accuracy 0.980
Epoch 0, Iteration 579, loss 0.017, batch accuracy 1.000
Epoch 0, Iteration 580, loss 0.015, batch accuracy 1.000
Epoch 0, Iteration 581, loss 0.008, batch accuracy 1.000
Epoch 0, Iteration 582, loss 0.018, batch accuracy 1.000
Epoch 0, Iteration 583, loss 0.050, batch accuracy 0.980
Epoch 0, Iteration 584, loss 0.137, batch accuracy 0.980
Epoch 0, Iteration 585, loss 0.036, batch accuracy 0.980
Epoch 0, Iteration 586, loss 0.025, batch accuracy 1.000
Epoch 0, Iteration 587, loss 0.105, batch accuracy 0.980
Epoch 0, Iteration 588, loss 0.065, batch accuracy 0.980
Epoch 0, Iteration 589, loss 0.080, batch accuracy 0.980
Epoch 0, Iteration 590, loss 0.052, batch accuracy 0.980
Epoch 0, Iteration 591, loss 0.017, batch accuracy 1.000
Epoch 0, Iteration 592, loss 0.022, batch accuracy 1.000
Epoch 0, Iteration 593, loss 0.061, batch accuracy 0.980
Epoch 0, Iteration 594, loss 0.045, batch accuracy 0.980
Epoch 0, Iteration 595, loss 0.

...

Epoch 1, Iteration 532, loss 0.014, batch accuracy 1.000
Epoch 1, Iteration 533, loss 0.183, batch accuracy 0.980
Epoch 1, Iteration 534, loss 0.079, batch accuracy 0.980
Epoch 1, Iteration 535, loss 0.077, batch accuracy 0.980
Epoch 1, Iteration 536, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 537, loss 0.008, batch accuracy 1.000
Epoch 1, Iteration 538, loss 0.014, batch accuracy 1.000
Epoch 1, Iteration 539, loss 0.108, batch accuracy 0.980
Epoch 1, Iteration 540, loss 0.024, batch accuracy 1.000
Epoch 1, Iteration 541, loss 0.101, batch accuracy 0.960
Epoch 1, Iteration 542, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 543, loss 0.044, batch accuracy 0.980
Epoch 1, Iteration 544, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 545, loss 0.083, batch accuracy 0.980
Epoch 1, Iteration 546, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 547, loss 0.060, batch accuracy 0.980
Epoch 1, Iteration 548, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 549, loss 0.

Epoch 1, Iteration 676, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 677, loss 0.035, batch accuracy 0.980
Epoch 1, Iteration 678, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 679, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 680, loss 0.014, batch accuracy 1.000
Epoch 1, Iteration 681, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 682, loss 0.019, batch accuracy 1.000
Epoch 1, Iteration 683, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 684, loss 0.056, batch accuracy 0.980
Epoch 1, Iteration 685, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 686, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 687, loss 0.107, batch accuracy 0.980
Epoch 1, Iteration 688, loss 0.048, batch accuracy 0.960
Epoch 1, Iteration 689, loss 0.096, batch accuracy 0.980
Epoch 1, Iteration 690, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 691, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 692, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 693, loss 0.

Epoch 1, Iteration 820, loss 0.089, batch accuracy 0.960
Epoch 1, Iteration 821, loss 0.210, batch accuracy 0.940
Epoch 1, Iteration 822, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 823, loss 0.031, batch accuracy 0.980
Epoch 1, Iteration 824, loss 0.014, batch accuracy 1.000
Epoch 1, Iteration 825, loss 0.088, batch accuracy 0.940
Epoch 1, Iteration 826, loss 0.098, batch accuracy 0.960
Epoch 1, Iteration 827, loss 0.026, batch accuracy 1.000
Epoch 1, Iteration 828, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 829, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 830, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 831, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 832, loss 0.024, batch accuracy 1.000
Epoch 1, Iteration 833, loss 0.135, batch accuracy 0.980
Epoch 1, Iteration 834, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 835, loss 0.192, batch accuracy 0.940
Epoch 1, Iteration 836, loss 0.095, batch accuracy 0.960
Epoch 1, Iteration 837, loss 0.

Epoch 1, Iteration 964, loss 0.033, batch accuracy 0.980
Epoch 1, Iteration 965, loss 0.027, batch accuracy 0.980
Epoch 1, Iteration 966, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 967, loss 0.055, batch accuracy 0.980
Epoch 1, Iteration 968, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 969, loss 0.023, batch accuracy 0.980
Epoch 1, Iteration 970, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 971, loss 0.071, batch accuracy 0.960
Epoch 1, Iteration 972, loss 0.015, batch accuracy 1.000
Epoch 1, Iteration 973, loss 0.066, batch accuracy 0.980
Epoch 1, Iteration 974, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 975, loss 0.013, batch accuracy 1.000
Epoch 1, Iteration 976, loss 0.058, batch accuracy 0.980
Epoch 1, Iteration 977, loss 0.014, batch accuracy 1.000
Epoch 1, Iteration 978, loss 0.061, batch accuracy 0.980
Epoch 1, Iteration 979, loss 0.027, batch accuracy 1.000
Epoch 1, Iteration 980, loss 0.022, batch accuracy 1.000
Epoch 1, Iteration 981, loss 0.

Epoch 1, Iteration 1107, loss 0.020, batch accuracy 0.980
Epoch 1, Iteration 1108, loss 0.074, batch accuracy 0.980
Epoch 1, Iteration 1109, loss 0.127, batch accuracy 0.960
Epoch 1, Iteration 1110, loss 0.019, batch accuracy 1.000
Epoch 1, Iteration 1111, loss 0.036, batch accuracy 0.980
Epoch 1, Iteration 1112, loss 0.099, batch accuracy 0.980
Epoch 1, Iteration 1113, loss 0.018, batch accuracy 0.980
Epoch 1, Iteration 1114, loss 0.013, batch accuracy 1.000
Epoch 1, Iteration 1115, loss 0.050, batch accuracy 0.980
Epoch 1, Iteration 1116, loss 0.186, batch accuracy 0.980
Epoch 1, Iteration 1117, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 1118, loss 0.045, batch accuracy 0.980
Epoch 1, Iteration 1119, loss 0.024, batch accuracy 1.000
Epoch 1, Iteration 1120, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 1121, loss 0.099, batch accuracy 0.960
Epoch 1, Iteration 1122, loss 0.051, batch accuracy 0.980
Epoch 1, Iteration 1123, loss 0.013, batch accuracy 1.000
Epoch 1, Itera

we loop over the test dataset and compute the corresponding classification accuracy. 

- Similar to Task 3.1, include a figure to visualize your training & testing(see below) performances during iterations and discuss your observations in your report. 

In [11]:
start_time = timeit.default_timer()

batch_size_test = 20
nb_test_points = x_test.shape[0] 
nb_iterations = nb_test_points//batch_size_test
preds = []
for i in range(nb_iterations):
    input_x_test = x_test[i*batch_size_test: (i+1)*batch_size_test]
    input_x_test = input_x_test[:, :, :,np.newaxis]
    preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
    preds.append(np.argmax(preds_test, axis=1))
    if np.mod(nb_test_points, batch_size_test) !=0:
        input_x_test = x_test[i*batch_size_test: -1]
        preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
        preds.append(np.argmax(preds, axis=1))
all_preds = np.concatenate(preds, axis =0)
y_real = np.argmax(y_test, axis=1)
acc_test = np.mean((all_preds==y_real)*1)
print('Test accuracy achieved: %.3f' %acc_test)

cnn_acc.append(acc_test)

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Test accuracy achieved: 0.990
Runtime:  10.870628777000036  seconds


##### Task 4.2
Similar to task 3.2, discuss 
- Plot a graph showing complexity (number of paramters) vs. accuracy vs. depth of networks. For this part you need to train four additional CNNs of different depths/widths (again, your choice) and report the results in a Table. Discuss the results and provide conclusion.


- In addition discuss and analyze the differences in term of performance, number of model parameters (i.e. weights/biases) and training/testing times between CNNs and MLPs. Provide a concusion. (For these discussions you should compare your results in Table 4.2 to Table 3.2)

#### CNN 2: 2 HL filters[32, 64] k(4x4)

In [12]:
class CNN2:
    def __init__(self, x_train, y_train, output_dir, lr=0.001, nb_epochs=10, batch_size=50):
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.batch_size = batch_size
        self.nb_epochs = nb_epochs
        self.nb_images, self.edge, _ = x_train.shape
        self.nb_iterations = self.nb_images // batch_size
        self.output_dir = output_dir
        self.im = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.labels = tf.placeholder(tf.float32, [None, 10])
        self.x_train = x_train[:,:,:, np.newaxis]
        self.y_train = y_train
        
    def create_model(self):
        with tf.variable_scope('CNN2', reuse=tf.AUTO_REUSE):
             
            hl_1 = tf.layers.conv2d(self.im, 32, kernel_size=(4, 4), strides=1, activation=tf.nn.relu, name='hl_1')
            hl_2 = tf.layers.conv2d(hl_1, 64, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_2')
            
            self.logits = tf.layers.dense(tf.layers.flatten(hl_2),10,activation=None, name='logits')
            self.preds = tf.nn.softmax(self.logits, name='preds')
            
    
    def compute_loss(self):
        with tf.variable_scope('loss'):
        
#             self.loss = tf.losses.softmax_cross_entropy(self.labels, self.logits)
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.labels,logits=self.logits))
            #########################################
            self.loss_summ = tf.summary.scalar("softmax_loss", self.loss)
    def optimizer(self):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5)
            self.model_vars = tf.trainable_variables()
            self.trainer = optimizer.minimize(self.loss, var_list=self.model_vars)

In [13]:
edge = int(np.sqrt(nb_features))
x_train.resize([n_train, edge, edge])
x_test.resize([n_test, edge, edge])

In [14]:
model = CNN2(x_train, y_train, './CNN2_logdir/', 0.001, 2, 50)
model.create_model()
model.compute_loss()
model.optimizer()
init = (tf.global_variables_initializer(),
        tf.local_variables_initializer())

saver = tf.train.Saver()
summary =tf.Summary()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.summary.FileWriter(model.output_dir)
writer.add_graph(sess.graph)
if not os.path.exists(model.output_dir):
    os.makedirs(model.output_dir)

In [15]:
start_time = timeit.default_timer()

for epoch in range(model.nb_epochs):
    randomize = np.arange(x_train.shape[0])
    np.random.shuffle(randomize)
    x_in = model.x_train[randomize,:]
    y_in = model.y_train[randomize,:]
    for i in range(model.nb_iterations):
        input_x_train = x_in[i*model.batch_size: (i+1)*model.batch_size]
        input_y_train = y_in[i*model.batch_size: (i+1)*model.batch_size]
        _ , preds, loss, loss_summ = sess.run([model.trainer, model.preds, model.loss, model.loss_summ], 
                                 feed_dict={model.im: input_x_train, 
                                            model.labels: input_y_train})
        y_preds = np.argmax(preds, axis=1)
        y_real = np.argmax(input_y_train, axis=1)
        acc_train = np.mean((y_preds==y_real)*1)
        print('Epoch %d, Iteration %d, loss %.3f, batch accuracy %.3f' %(epoch, i, loss, acc_train))
        writer.add_summary(loss_summ, epoch * model.nb_iterations + i)
    saver.save(sess, model.output_dir, global_step=epoch)  
# sess.close()

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Epoch 0, Iteration 0, loss 2.351, batch accuracy 0.020
Epoch 0, Iteration 1, loss 2.339, batch accuracy 0.120
Epoch 0, Iteration 2, loss 2.215, batch accuracy 0.240
Epoch 0, Iteration 3, loss 2.158, batch accuracy 0.280
Epoch 0, Iteration 4, loss 2.095, batch accuracy 0.300
Epoch 0, Iteration 5, loss 1.841, batch accuracy 0.560
Epoch 0, Iteration 6, loss 1.627, batch accuracy 0.580
Epoch 0, Iteration 7, loss 1.689, batch accuracy 0.340
Epoch 0, Iteration 8, loss 1.331, batch accuracy 0.760
Epoch 0, Iteration 9, loss 1.548, batch accuracy 0.420
Epoch 0, Iteration 10, loss 1.403, batch accuracy 0.460
Epoch 0, Iteration 11, loss 1.147, batch accuracy 0.680
Epoch 0, Iteration 12, loss 1.208, batch accuracy 0.620
Epoch 0, Iteration 13, loss 0.942, batch accuracy 0.720
Epoch 0, Iteration 14, loss 0.833, batch accuracy 0.820
Epoch 0, Iteration 15, loss 0.861, batch accuracy 0.780
Epoch 0, Iteration 16, loss 0.652, batch accuracy 0.820
Epoch 0, Iteration 17, loss 0.621, batch accuracy 0.780
Ep

Epoch 0, Iteration 146, loss 0.143, batch accuracy 0.960
Epoch 0, Iteration 147, loss 0.137, batch accuracy 0.980
Epoch 0, Iteration 148, loss 0.224, batch accuracy 0.980
Epoch 0, Iteration 149, loss 0.144, batch accuracy 0.940
Epoch 0, Iteration 150, loss 0.138, batch accuracy 0.960
Epoch 0, Iteration 151, loss 0.156, batch accuracy 0.960
Epoch 0, Iteration 152, loss 0.148, batch accuracy 0.960
Epoch 0, Iteration 153, loss 0.097, batch accuracy 0.980
Epoch 0, Iteration 154, loss 0.230, batch accuracy 0.940
Epoch 0, Iteration 155, loss 0.140, batch accuracy 0.940
Epoch 0, Iteration 156, loss 0.096, batch accuracy 0.960
Epoch 0, Iteration 157, loss 0.043, batch accuracy 1.000
Epoch 0, Iteration 158, loss 0.161, batch accuracy 0.920
Epoch 0, Iteration 159, loss 0.224, batch accuracy 0.920
Epoch 0, Iteration 160, loss 0.216, batch accuracy 0.940
Epoch 0, Iteration 161, loss 0.081, batch accuracy 0.960
Epoch 0, Iteration 162, loss 0.153, batch accuracy 0.940
Epoch 0, Iteration 163, loss 0.

Epoch 0, Iteration 290, loss 0.152, batch accuracy 0.960
Epoch 0, Iteration 291, loss 0.186, batch accuracy 0.920
Epoch 0, Iteration 292, loss 0.207, batch accuracy 0.960
Epoch 0, Iteration 293, loss 0.055, batch accuracy 0.980
Epoch 0, Iteration 294, loss 0.149, batch accuracy 0.960
Epoch 0, Iteration 295, loss 0.138, batch accuracy 0.940
Epoch 0, Iteration 296, loss 0.062, batch accuracy 0.980
Epoch 0, Iteration 297, loss 0.087, batch accuracy 0.980
Epoch 0, Iteration 298, loss 0.115, batch accuracy 0.960
Epoch 0, Iteration 299, loss 0.170, batch accuracy 0.960
Epoch 0, Iteration 300, loss 0.167, batch accuracy 0.940
Epoch 0, Iteration 301, loss 0.189, batch accuracy 0.940
Epoch 0, Iteration 302, loss 0.178, batch accuracy 0.940
Epoch 0, Iteration 303, loss 0.300, batch accuracy 0.940
Epoch 0, Iteration 304, loss 0.021, batch accuracy 1.000
Epoch 0, Iteration 305, loss 0.062, batch accuracy 0.980
Epoch 0, Iteration 306, loss 0.192, batch accuracy 0.940
Epoch 0, Iteration 307, loss 0.

Epoch 0, Iteration 434, loss 0.028, batch accuracy 1.000
Epoch 0, Iteration 435, loss 0.071, batch accuracy 0.980
Epoch 0, Iteration 436, loss 0.095, batch accuracy 0.960
Epoch 0, Iteration 437, loss 0.063, batch accuracy 0.960
Epoch 0, Iteration 438, loss 0.081, batch accuracy 0.980
Epoch 0, Iteration 439, loss 0.032, batch accuracy 1.000
Epoch 0, Iteration 440, loss 0.041, batch accuracy 1.000
Epoch 0, Iteration 441, loss 0.075, batch accuracy 0.960
Epoch 0, Iteration 442, loss 0.034, batch accuracy 1.000
Epoch 0, Iteration 443, loss 0.031, batch accuracy 0.980
Epoch 0, Iteration 444, loss 0.026, batch accuracy 1.000
Epoch 0, Iteration 445, loss 0.087, batch accuracy 0.960
Epoch 0, Iteration 446, loss 0.009, batch accuracy 1.000
Epoch 0, Iteration 447, loss 0.238, batch accuracy 0.920
Epoch 0, Iteration 448, loss 0.055, batch accuracy 0.980
Epoch 0, Iteration 449, loss 0.134, batch accuracy 0.980
Epoch 0, Iteration 450, loss 0.038, batch accuracy 1.000
Epoch 0, Iteration 451, loss 0.

Epoch 0, Iteration 578, loss 0.062, batch accuracy 1.000
Epoch 0, Iteration 579, loss 0.047, batch accuracy 0.980
Epoch 0, Iteration 580, loss 0.068, batch accuracy 0.980
Epoch 0, Iteration 581, loss 0.133, batch accuracy 0.980
Epoch 0, Iteration 582, loss 0.025, batch accuracy 1.000
Epoch 0, Iteration 583, loss 0.047, batch accuracy 0.980
Epoch 0, Iteration 584, loss 0.031, batch accuracy 0.980
Epoch 0, Iteration 585, loss 0.023, batch accuracy 1.000
Epoch 0, Iteration 586, loss 0.179, batch accuracy 0.960
Epoch 0, Iteration 587, loss 0.208, batch accuracy 0.920
Epoch 0, Iteration 588, loss 0.016, batch accuracy 1.000
Epoch 0, Iteration 589, loss 0.104, batch accuracy 0.960
Epoch 0, Iteration 590, loss 0.305, batch accuracy 0.940
Epoch 0, Iteration 591, loss 0.038, batch accuracy 1.000
Epoch 0, Iteration 592, loss 0.033, batch accuracy 1.000
Epoch 0, Iteration 593, loss 0.185, batch accuracy 0.920
Epoch 0, Iteration 594, loss 0.198, batch accuracy 0.940
Epoch 0, Iteration 595, loss 0.

...

Epoch 1, Iteration 532, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 533, loss 0.018, batch accuracy 1.000
Epoch 1, Iteration 534, loss 0.070, batch accuracy 0.960
Epoch 1, Iteration 535, loss 0.029, batch accuracy 0.980
Epoch 1, Iteration 536, loss 0.058, batch accuracy 0.980
Epoch 1, Iteration 537, loss 0.101, batch accuracy 0.980
Epoch 1, Iteration 538, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 539, loss 0.038, batch accuracy 1.000
Epoch 1, Iteration 540, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 541, loss 0.043, batch accuracy 0.980
Epoch 1, Iteration 542, loss 0.078, batch accuracy 0.960
Epoch 1, Iteration 543, loss 0.060, batch accuracy 0.960
Epoch 1, Iteration 544, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 545, loss 0.008, batch accuracy 1.000
Epoch 1, Iteration 546, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 547, loss 0.032, batch accuracy 0.980
Epoch 1, Iteration 548, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 549, loss 0.

Epoch 1, Iteration 676, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 677, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 678, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 679, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 680, loss 0.019, batch accuracy 1.000
Epoch 1, Iteration 681, loss 0.019, batch accuracy 1.000
Epoch 1, Iteration 682, loss 0.065, batch accuracy 0.960
Epoch 1, Iteration 683, loss 0.025, batch accuracy 1.000
Epoch 1, Iteration 684, loss 0.008, batch accuracy 1.000
Epoch 1, Iteration 685, loss 0.045, batch accuracy 0.960
Epoch 1, Iteration 686, loss 0.106, batch accuracy 0.980
Epoch 1, Iteration 687, loss 0.041, batch accuracy 0.980
Epoch 1, Iteration 688, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 689, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 690, loss 0.061, batch accuracy 0.980
Epoch 1, Iteration 691, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 692, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 693, loss 0.

Epoch 1, Iteration 820, loss 0.021, batch accuracy 1.000
Epoch 1, Iteration 821, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 822, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 823, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 824, loss 0.065, batch accuracy 0.980
Epoch 1, Iteration 825, loss 0.020, batch accuracy 1.000
Epoch 1, Iteration 826, loss 0.165, batch accuracy 0.960
Epoch 1, Iteration 827, loss 0.015, batch accuracy 1.000
Epoch 1, Iteration 828, loss 0.072, batch accuracy 0.940
Epoch 1, Iteration 829, loss 0.064, batch accuracy 0.980
Epoch 1, Iteration 830, loss 0.083, batch accuracy 0.980
Epoch 1, Iteration 831, loss 0.109, batch accuracy 0.980
Epoch 1, Iteration 832, loss 0.045, batch accuracy 0.980
Epoch 1, Iteration 833, loss 0.156, batch accuracy 0.980
Epoch 1, Iteration 834, loss 0.043, batch accuracy 1.000
Epoch 1, Iteration 835, loss 0.293, batch accuracy 0.960
Epoch 1, Iteration 836, loss 0.098, batch accuracy 0.980
Epoch 1, Iteration 837, loss 0.

Epoch 1, Iteration 964, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 965, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 966, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 967, loss 0.008, batch accuracy 1.000
Epoch 1, Iteration 968, loss 0.021, batch accuracy 1.000
Epoch 1, Iteration 969, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 970, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 971, loss 0.021, batch accuracy 0.980
Epoch 1, Iteration 972, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 973, loss 0.046, batch accuracy 0.980
Epoch 1, Iteration 974, loss 0.133, batch accuracy 0.980
Epoch 1, Iteration 975, loss 0.110, batch accuracy 0.980
Epoch 1, Iteration 976, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 977, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 978, loss 0.025, batch accuracy 0.980
Epoch 1, Iteration 979, loss 0.190, batch accuracy 0.960
Epoch 1, Iteration 980, loss 0.044, batch accuracy 0.980
Epoch 1, Iteration 981, loss 0.

Epoch 1, Iteration 1107, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 1108, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 1109, loss 0.030, batch accuracy 1.000
Epoch 1, Iteration 1110, loss 0.042, batch accuracy 0.980
Epoch 1, Iteration 1111, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 1112, loss 0.058, batch accuracy 0.980
Epoch 1, Iteration 1113, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 1114, loss 0.141, batch accuracy 0.940
Epoch 1, Iteration 1115, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 1116, loss 0.024, batch accuracy 0.980
Epoch 1, Iteration 1117, loss 0.110, batch accuracy 0.960
Epoch 1, Iteration 1118, loss 0.160, batch accuracy 0.960
Epoch 1, Iteration 1119, loss 0.015, batch accuracy 1.000
Epoch 1, Iteration 1120, loss 0.022, batch accuracy 0.980
Epoch 1, Iteration 1121, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 1122, loss 0.042, batch accuracy 0.980
Epoch 1, Iteration 1123, loss 0.017, batch accuracy 1.000
Epoch 1, Itera

In [16]:
start_time = timeit.default_timer()

batch_size_test = 20
nb_test_points = x_test.shape[0] 
nb_iterations = nb_test_points//batch_size_test
preds = []
for i in range(nb_iterations):
    input_x_test = x_test[i*batch_size_test: (i+1)*batch_size_test]
    input_x_test = input_x_test[:, :, :,np.newaxis]
    preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
    preds.append(np.argmax(preds_test, axis=1))
    if np.mod(nb_test_points, batch_size_test) !=0:
        input_x_test = x_test[i*batch_size_test: -1]
        preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
        preds.append(np.argmax(preds, axis=1))
all_preds = np.concatenate(preds, axis =0)
y_real = np.argmax(y_test, axis=1)
acc_test = np.mean((all_preds==y_real)*1)
print('Test accuracy achieved: %.3f' %acc_test)

cnn_acc.append(acc_test)

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Test accuracy achieved: 0.988
Runtime:  6.608966227999986  seconds


#### CNN 3: 4 HL filters[32, 64, 128, 256] k(8x8) s[1,2,2,2]

In [17]:
class CNN3_x:
    def __init__(self, x_train, y_train, output_dir, lr=0.001, nb_epochs=10, batch_size=50):
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.batch_size = batch_size
        self.nb_epochs = nb_epochs
        self.nb_images, self.edge, _ = x_train.shape
        self.nb_iterations = self.nb_images // batch_size
        self.output_dir = output_dir
        self.im = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.labels = tf.placeholder(tf.float32, [None, 10])
        self.x_train = x_train[:,:,:, np.newaxis]
        self.y_train = y_train
        
    def create_model(self):
        with tf.variable_scope('CNN3_x', reuse=tf.AUTO_REUSE):
             
            hl_1 = tf.layers.conv2d(self.im, 32, kernel_size=(4, 4), strides=1, activation=tf.nn.relu, name='hl_1')
            hl_2 = tf.layers.conv2d(hl_1, 64, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_2')
            hl_3 = tf.layers.conv2d(hl_2, 128, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_3')
            hl_4 = tf.layers.conv2d(hl_3, 256, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_4')

            self.logits = tf.layers.dense(tf.layers.flatten(hl_4),10,activation=None, name='logits')
            self.preds = tf.nn.softmax(self.logits, name='preds')
            
    
    def compute_loss(self):
        with tf.variable_scope('loss'):
#             self.loss = tf.losses.softmax_cross_entropy(self.labels, self.logits)
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.labels,logits=self.logits))
            #########################################
            self.loss_summ = tf.summary.scalar("softmax_loss", self.loss)
    def optimizer(self):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5)
            self.model_vars = tf.trainable_variables()
            self.trainer = optimizer.minimize(self.loss, var_list=self.model_vars)

In [18]:
edge = int(np.sqrt(nb_features))
x_train.resize([n_train, edge, edge])
x_test.resize([n_test, edge, edge])

In [19]:
model = CNN3_x(x_train, y_train, './CNN3_x_logdir/', 0.001, 2, 50)
model.create_model()
model.compute_loss()
model.optimizer()
init = (tf.global_variables_initializer(),
        tf.local_variables_initializer())

saver = tf.train.Saver()
summary =tf.Summary()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.summary.FileWriter(model.output_dir)
writer.add_graph(sess.graph)
if not os.path.exists(model.output_dir):
    os.makedirs(model.output_dir)

In [20]:
start_time = timeit.default_timer()

for epoch in range(model.nb_epochs):
    randomize = np.arange(x_train.shape[0])
    np.random.shuffle(randomize)
    x_in = model.x_train[randomize,:]
    y_in = model.y_train[randomize,:]
    for i in range(model.nb_iterations):
        input_x_train = x_in[i*model.batch_size: (i+1)*model.batch_size]
        input_y_train = y_in[i*model.batch_size: (i+1)*model.batch_size]
        _ , preds, loss, loss_summ = sess.run([model.trainer, model.preds, model.loss, model.loss_summ], 
                                 feed_dict={model.im: input_x_train, 
                                            model.labels: input_y_train})
        y_preds = np.argmax(preds, axis=1)
        y_real = np.argmax(input_y_train, axis=1)
        acc_train = np.mean((y_preds==y_real)*1)
        print('Epoch %d, Iteration %d, loss %.3f, batch accuracy %.3f' %(epoch, i, loss, acc_train))
        writer.add_summary(loss_summ, epoch * model.nb_iterations + i)
    saver.save(sess, model.output_dir, global_step=epoch)  
# sess.close()

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Epoch 0, Iteration 0, loss 2.303, batch accuracy 0.080
Epoch 0, Iteration 1, loss 2.283, batch accuracy 0.120
Epoch 0, Iteration 2, loss 2.218, batch accuracy 0.300
Epoch 0, Iteration 3, loss 2.190, batch accuracy 0.420
Epoch 0, Iteration 4, loss 2.013, batch accuracy 0.560
Epoch 0, Iteration 5, loss 1.815, batch accuracy 0.400
Epoch 0, Iteration 6, loss 1.516, batch accuracy 0.540
Epoch 0, Iteration 7, loss 1.789, batch accuracy 0.400
Epoch 0, Iteration 8, loss 1.763, batch accuracy 0.480
Epoch 0, Iteration 9, loss 1.323, batch accuracy 0.620
Epoch 0, Iteration 10, loss 1.053, batch accuracy 0.720
Epoch 0, Iteration 11, loss 0.899, batch accuracy 0.700
Epoch 0, Iteration 12, loss 0.858, batch accuracy 0.760
Epoch 0, Iteration 13, loss 0.512, batch accuracy 0.800
Epoch 0, Iteration 14, loss 0.755, batch accuracy 0.720
Epoch 0, Iteration 15, loss 0.779, batch accuracy 0.720
Epoch 0, Iteration 16, loss 0.761, batch accuracy 0.780
Epoch 0, Iteration 17, loss 0.726, batch accuracy 0.780
Ep

Epoch 0, Iteration 146, loss 0.274, batch accuracy 0.900
Epoch 0, Iteration 147, loss 0.055, batch accuracy 0.980
Epoch 0, Iteration 148, loss 0.098, batch accuracy 0.960
Epoch 0, Iteration 149, loss 0.123, batch accuracy 0.960
Epoch 0, Iteration 150, loss 0.027, batch accuracy 1.000
Epoch 0, Iteration 151, loss 0.157, batch accuracy 0.940
Epoch 0, Iteration 152, loss 0.154, batch accuracy 0.940
Epoch 0, Iteration 153, loss 0.172, batch accuracy 0.940
Epoch 0, Iteration 154, loss 0.167, batch accuracy 0.960
Epoch 0, Iteration 155, loss 0.050, batch accuracy 0.980
Epoch 0, Iteration 156, loss 0.064, batch accuracy 0.980
Epoch 0, Iteration 157, loss 0.140, batch accuracy 0.960
Epoch 0, Iteration 158, loss 0.049, batch accuracy 0.980
Epoch 0, Iteration 159, loss 0.030, batch accuracy 1.000
Epoch 0, Iteration 160, loss 0.182, batch accuracy 0.980
Epoch 0, Iteration 161, loss 0.202, batch accuracy 0.960
Epoch 0, Iteration 162, loss 0.061, batch accuracy 0.980
Epoch 0, Iteration 163, loss 0.

Epoch 0, Iteration 290, loss 0.089, batch accuracy 0.980
Epoch 0, Iteration 291, loss 0.052, batch accuracy 1.000
Epoch 0, Iteration 292, loss 0.064, batch accuracy 0.980
Epoch 0, Iteration 293, loss 0.205, batch accuracy 0.920
Epoch 0, Iteration 294, loss 0.106, batch accuracy 0.980
Epoch 0, Iteration 295, loss 0.160, batch accuracy 0.960
Epoch 0, Iteration 296, loss 0.122, batch accuracy 0.960
Epoch 0, Iteration 297, loss 0.095, batch accuracy 0.980
Epoch 0, Iteration 298, loss 0.283, batch accuracy 0.920
Epoch 0, Iteration 299, loss 0.115, batch accuracy 0.940
Epoch 0, Iteration 300, loss 0.175, batch accuracy 0.940
Epoch 0, Iteration 301, loss 0.087, batch accuracy 0.980
Epoch 0, Iteration 302, loss 0.110, batch accuracy 0.980
Epoch 0, Iteration 303, loss 0.045, batch accuracy 0.980
Epoch 0, Iteration 304, loss 0.041, batch accuracy 0.980
Epoch 0, Iteration 305, loss 0.022, batch accuracy 1.000
Epoch 0, Iteration 306, loss 0.236, batch accuracy 0.920
Epoch 0, Iteration 307, loss 0.

Epoch 0, Iteration 434, loss 0.179, batch accuracy 0.960
Epoch 0, Iteration 435, loss 0.071, batch accuracy 0.980
Epoch 0, Iteration 436, loss 0.035, batch accuracy 1.000
Epoch 0, Iteration 437, loss 0.117, batch accuracy 0.940
Epoch 0, Iteration 438, loss 0.054, batch accuracy 1.000
Epoch 0, Iteration 439, loss 0.170, batch accuracy 0.960
Epoch 0, Iteration 440, loss 0.021, batch accuracy 1.000
Epoch 0, Iteration 441, loss 0.193, batch accuracy 0.960
Epoch 0, Iteration 442, loss 0.039, batch accuracy 1.000
Epoch 0, Iteration 443, loss 0.023, batch accuracy 1.000
Epoch 0, Iteration 444, loss 0.277, batch accuracy 0.940
Epoch 0, Iteration 445, loss 0.035, batch accuracy 1.000
Epoch 0, Iteration 446, loss 0.088, batch accuracy 0.960
Epoch 0, Iteration 447, loss 0.178, batch accuracy 0.960
Epoch 0, Iteration 448, loss 0.159, batch accuracy 0.960
Epoch 0, Iteration 449, loss 0.126, batch accuracy 0.960
Epoch 0, Iteration 450, loss 0.175, batch accuracy 0.940
Epoch 0, Iteration 451, loss 0.

Epoch 0, Iteration 578, loss 0.056, batch accuracy 0.980
Epoch 0, Iteration 579, loss 0.008, batch accuracy 1.000
Epoch 0, Iteration 580, loss 0.023, batch accuracy 1.000
Epoch 0, Iteration 581, loss 0.034, batch accuracy 1.000
Epoch 0, Iteration 582, loss 0.095, batch accuracy 0.980
Epoch 0, Iteration 583, loss 0.018, batch accuracy 1.000
Epoch 0, Iteration 584, loss 0.083, batch accuracy 0.980
Epoch 0, Iteration 585, loss 0.050, batch accuracy 0.980
Epoch 0, Iteration 586, loss 0.013, batch accuracy 1.000
Epoch 0, Iteration 587, loss 0.140, batch accuracy 0.960
Epoch 0, Iteration 588, loss 0.015, batch accuracy 1.000
Epoch 0, Iteration 589, loss 0.005, batch accuracy 1.000
Epoch 0, Iteration 590, loss 0.030, batch accuracy 1.000
Epoch 0, Iteration 591, loss 0.076, batch accuracy 0.980
Epoch 0, Iteration 592, loss 0.025, batch accuracy 1.000
Epoch 0, Iteration 593, loss 0.007, batch accuracy 1.000
Epoch 0, Iteration 594, loss 0.077, batch accuracy 0.980
Epoch 0, Iteration 595, loss 0.

...

Epoch 1, Iteration 531, loss 0.169, batch accuracy 0.960
Epoch 1, Iteration 532, loss 0.092, batch accuracy 0.960
Epoch 1, Iteration 533, loss 0.094, batch accuracy 0.960
Epoch 1, Iteration 534, loss 0.067, batch accuracy 0.980
Epoch 1, Iteration 535, loss 0.038, batch accuracy 0.980
Epoch 1, Iteration 536, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 537, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 538, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 539, loss 0.040, batch accuracy 0.980
Epoch 1, Iteration 540, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 541, loss 0.128, batch accuracy 0.960
Epoch 1, Iteration 542, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 543, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 544, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 545, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 546, loss 0.013, batch accuracy 1.000
Epoch 1, Iteration 547, loss 0.152, batch accuracy 0.960
Epoch 1, Iteration 548, loss 0.

Epoch 1, Iteration 675, loss 0.020, batch accuracy 0.980
Epoch 1, Iteration 676, loss 0.027, batch accuracy 0.980
Epoch 1, Iteration 677, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 678, loss 0.145, batch accuracy 0.960
Epoch 1, Iteration 679, loss 0.115, batch accuracy 0.960
Epoch 1, Iteration 680, loss 0.057, batch accuracy 0.980
Epoch 1, Iteration 681, loss 0.005, batch accuracy 1.000
Epoch 1, Iteration 682, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 683, loss 0.072, batch accuracy 0.980
Epoch 1, Iteration 684, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 685, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 686, loss 0.027, batch accuracy 0.980
Epoch 1, Iteration 687, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 688, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 689, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 690, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 691, loss 0.032, batch accuracy 0.980
Epoch 1, Iteration 692, loss 0.

Epoch 1, Iteration 819, loss 0.016, batch accuracy 0.980
Epoch 1, Iteration 820, loss 0.064, batch accuracy 0.980
Epoch 1, Iteration 821, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 822, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 823, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 824, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 825, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 826, loss 0.014, batch accuracy 1.000
Epoch 1, Iteration 827, loss 0.030, batch accuracy 0.980
Epoch 1, Iteration 828, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 829, loss 0.024, batch accuracy 0.980
Epoch 1, Iteration 830, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 831, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 832, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 833, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 834, loss 0.111, batch accuracy 0.960
Epoch 1, Iteration 835, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 836, loss 0.

Epoch 1, Iteration 963, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 964, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 965, loss 0.028, batch accuracy 0.980
Epoch 1, Iteration 966, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 967, loss 0.043, batch accuracy 0.980
Epoch 1, Iteration 968, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 969, loss 0.051, batch accuracy 0.980
Epoch 1, Iteration 970, loss 0.025, batch accuracy 0.980
Epoch 1, Iteration 971, loss 0.078, batch accuracy 0.980
Epoch 1, Iteration 972, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 973, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 974, loss 0.116, batch accuracy 0.960
Epoch 1, Iteration 975, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 976, loss 0.104, batch accuracy 0.960
Epoch 1, Iteration 977, loss 0.084, batch accuracy 0.980
Epoch 1, Iteration 978, loss 0.032, batch accuracy 0.980
Epoch 1, Iteration 979, loss 0.022, batch accuracy 1.000
Epoch 1, Iteration 980, loss 0.

Epoch 1, Iteration 1105, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 1106, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 1107, loss 0.020, batch accuracy 0.980
Epoch 1, Iteration 1108, loss 0.024, batch accuracy 1.000
Epoch 1, Iteration 1109, loss 0.024, batch accuracy 0.980
Epoch 1, Iteration 1110, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 1111, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 1112, loss 0.047, batch accuracy 0.980
Epoch 1, Iteration 1113, loss 0.274, batch accuracy 0.960
Epoch 1, Iteration 1114, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 1115, loss 0.019, batch accuracy 0.980
Epoch 1, Iteration 1116, loss 0.111, batch accuracy 0.960
Epoch 1, Iteration 1117, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 1118, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 1119, loss 0.045, batch accuracy 0.980
Epoch 1, Iteration 1120, loss 0.032, batch accuracy 0.980
Epoch 1, Iteration 1121, loss 0.064, batch accuracy 0.980
Epoch 1, Itera

In [21]:
start_time = timeit.default_timer()

batch_size_test = 20
nb_test_points = x_test.shape[0] 
nb_iterations = nb_test_points//batch_size_test
preds = []
for i in range(nb_iterations):
    input_x_test = x_test[i*batch_size_test: (i+1)*batch_size_test]
    input_x_test = input_x_test[:, :, :,np.newaxis]
    preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
    preds.append(np.argmax(preds_test, axis=1))
    if np.mod(nb_test_points, batch_size_test) !=0:
        input_x_test = x_test[i*batch_size_test: -1]
        preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
        preds.append(np.argmax(preds, axis=1))
all_preds = np.concatenate(preds, axis =0)
y_real = np.argmax(y_test, axis=1)
acc_test = np.mean((all_preds==y_real)*1)
print('Test accuracy achieved: %.3f' %acc_test)

cnn_acc.append(acc_test)

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Test accuracy achieved: 0.989
Runtime:  13.425617252999928  seconds


#### CNN 4: 2 HL filters[256,512] k(4x4) s[2,2]

In [22]:
class CNN4:
    def __init__(self, x_train, y_train, output_dir, lr=0.001, nb_epochs=10, batch_size=50):
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.batch_size = batch_size
        self.nb_epochs = nb_epochs
        self.nb_images, self.edge, _ = x_train.shape
        self.nb_iterations = self.nb_images // batch_size
        self.output_dir = output_dir
        self.im = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.labels = tf.placeholder(tf.float32, [None, 10])
        self.x_train = x_train[:,:,:, np.newaxis]
        self.y_train = y_train
        
    def create_model(self):
        with tf.variable_scope('CNN4', reuse=tf.AUTO_REUSE):
            
            hl_1 = tf.layers.conv2d(self.im, 256, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_1')
            hl_2 = tf.layers.conv2d(hl_1, 512, kernel_size=(4, 4), strides=2, activation=tf.nn.relu, name='hl_2')
        
            self.logits = tf.layers.dense(tf.layers.flatten(hl_2),10,activation=None, name='logits')
            self.preds = tf.nn.softmax(self.logits, name='preds')
            
    
    def compute_loss(self):
        with tf.variable_scope('loss'):
#             self.loss = tf.losses.softmax_cross_entropy(self.labels, self.logits)
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.labels,logits=self.logits))
            #########################################
            self.loss_summ = tf.summary.scalar("softmax_loss", self.loss)
    def optimizer(self):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5)
            self.model_vars = tf.trainable_variables()
            self.trainer = optimizer.minimize(self.loss, var_list=self.model_vars)

In [23]:
edge = int(np.sqrt(nb_features))
x_train.resize([n_train, edge, edge])
x_test.resize([n_test, edge, edge])

In [24]:
model = CNN4(x_train, y_train, './CNN4_logdir/', 0.001, 2, 50)
model.create_model()
model.compute_loss()
model.optimizer()
init = (tf.global_variables_initializer(),
        tf.local_variables_initializer())

saver = tf.train.Saver()
summary =tf.Summary()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.summary.FileWriter(model.output_dir)
writer.add_graph(sess.graph)
if not os.path.exists(model.output_dir):
    os.makedirs(model.output_dir)

In [25]:
start_time = timeit.default_timer()

for epoch in range(model.nb_epochs):
    randomize = np.arange(x_train.shape[0])
    np.random.shuffle(randomize)
    x_in = model.x_train[randomize,:]
    y_in = model.y_train[randomize,:]
    for i in range(model.nb_iterations):
        input_x_train = x_in[i*model.batch_size: (i+1)*model.batch_size]
        input_y_train = y_in[i*model.batch_size: (i+1)*model.batch_size]
        _ , preds, loss, loss_summ = sess.run([model.trainer, model.preds, model.loss, model.loss_summ], 
                                 feed_dict={model.im: input_x_train, 
                                            model.labels: input_y_train})
        y_preds = np.argmax(preds, axis=1)
        y_real = np.argmax(input_y_train, axis=1)
        acc_train = np.mean((y_preds==y_real)*1)
        print('Epoch %d, Iteration %d, loss %.3f, batch accuracy %.3f' %(epoch, i, loss, acc_train))
        writer.add_summary(loss_summ, epoch * model.nb_iterations + i)
    saver.save(sess, model.output_dir, global_step=epoch)  
# sess.close()

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Epoch 0, Iteration 0, loss 2.309, batch accuracy 0.020
Epoch 0, Iteration 1, loss 2.436, batch accuracy 0.240
Epoch 0, Iteration 2, loss 2.068, batch accuracy 0.380
Epoch 0, Iteration 3, loss 2.087, batch accuracy 0.100
Epoch 0, Iteration 4, loss 1.787, batch accuracy 0.600
Epoch 0, Iteration 5, loss 1.856, batch accuracy 0.420
Epoch 0, Iteration 6, loss 1.428, batch accuracy 0.640
Epoch 0, Iteration 7, loss 1.344, batch accuracy 0.480
Epoch 0, Iteration 8, loss 1.198, batch accuracy 0.640
Epoch 0, Iteration 9, loss 0.911, batch accuracy 0.700
Epoch 0, Iteration 10, loss 0.650, batch accuracy 0.780
Epoch 0, Iteration 11, loss 0.726, batch accuracy 0.740
Epoch 0, Iteration 12, loss 0.758, batch accuracy 0.740
Epoch 0, Iteration 13, loss 0.504, batch accuracy 0.820
Epoch 0, Iteration 14, loss 0.510, batch accuracy 0.860
Epoch 0, Iteration 15, loss 0.453, batch accuracy 0.840
Epoch 0, Iteration 16, loss 0.398, batch accuracy 0.860
Epoch 0, Iteration 17, loss 0.422, batch accuracy 0.880
Ep

Epoch 0, Iteration 146, loss 0.102, batch accuracy 0.960
Epoch 0, Iteration 147, loss 0.258, batch accuracy 0.940
Epoch 0, Iteration 148, loss 0.150, batch accuracy 0.940
Epoch 0, Iteration 149, loss 0.180, batch accuracy 0.960
Epoch 0, Iteration 150, loss 0.250, batch accuracy 0.920
Epoch 0, Iteration 151, loss 0.225, batch accuracy 0.920
Epoch 0, Iteration 152, loss 0.187, batch accuracy 0.940
Epoch 0, Iteration 153, loss 0.094, batch accuracy 0.980
Epoch 0, Iteration 154, loss 0.030, batch accuracy 1.000
Epoch 0, Iteration 155, loss 0.162, batch accuracy 0.960
Epoch 0, Iteration 156, loss 0.115, batch accuracy 0.960
Epoch 0, Iteration 157, loss 0.089, batch accuracy 0.980
Epoch 0, Iteration 158, loss 0.080, batch accuracy 0.980
Epoch 0, Iteration 159, loss 0.091, batch accuracy 0.960
Epoch 0, Iteration 160, loss 0.120, batch accuracy 0.980
Epoch 0, Iteration 161, loss 0.066, batch accuracy 1.000
Epoch 0, Iteration 162, loss 0.072, batch accuracy 0.980
Epoch 0, Iteration 163, loss 0.

Epoch 0, Iteration 290, loss 0.052, batch accuracy 0.980
Epoch 0, Iteration 291, loss 0.033, batch accuracy 1.000
Epoch 0, Iteration 292, loss 0.078, batch accuracy 1.000
Epoch 0, Iteration 293, loss 0.108, batch accuracy 0.960
Epoch 0, Iteration 294, loss 0.062, batch accuracy 0.980
Epoch 0, Iteration 295, loss 0.130, batch accuracy 0.960
Epoch 0, Iteration 296, loss 0.100, batch accuracy 0.960
Epoch 0, Iteration 297, loss 0.022, batch accuracy 1.000
Epoch 0, Iteration 298, loss 0.120, batch accuracy 0.920
Epoch 0, Iteration 299, loss 0.043, batch accuracy 0.980
Epoch 0, Iteration 300, loss 0.276, batch accuracy 0.920
Epoch 0, Iteration 301, loss 0.184, batch accuracy 0.960
Epoch 0, Iteration 302, loss 0.026, batch accuracy 1.000
Epoch 0, Iteration 303, loss 0.038, batch accuracy 0.980
Epoch 0, Iteration 304, loss 0.069, batch accuracy 0.980
Epoch 0, Iteration 305, loss 0.034, batch accuracy 1.000
Epoch 0, Iteration 306, loss 0.039, batch accuracy 1.000
Epoch 0, Iteration 307, loss 0.

Epoch 0, Iteration 434, loss 0.028, batch accuracy 1.000
Epoch 0, Iteration 435, loss 0.132, batch accuracy 0.940
Epoch 0, Iteration 436, loss 0.044, batch accuracy 0.980
Epoch 0, Iteration 437, loss 0.123, batch accuracy 0.960
Epoch 0, Iteration 438, loss 0.007, batch accuracy 1.000
Epoch 0, Iteration 439, loss 0.049, batch accuracy 0.980
Epoch 0, Iteration 440, loss 0.169, batch accuracy 0.940
Epoch 0, Iteration 441, loss 0.149, batch accuracy 0.980
Epoch 0, Iteration 442, loss 0.123, batch accuracy 0.960
Epoch 0, Iteration 443, loss 0.041, batch accuracy 0.980
Epoch 0, Iteration 444, loss 0.139, batch accuracy 0.980
Epoch 0, Iteration 445, loss 0.150, batch accuracy 0.940
Epoch 0, Iteration 446, loss 0.075, batch accuracy 0.960
Epoch 0, Iteration 447, loss 0.054, batch accuracy 1.000
Epoch 0, Iteration 448, loss 0.040, batch accuracy 1.000
Epoch 0, Iteration 449, loss 0.018, batch accuracy 1.000
Epoch 0, Iteration 450, loss 0.013, batch accuracy 1.000
Epoch 0, Iteration 451, loss 0.

Epoch 0, Iteration 578, loss 0.013, batch accuracy 1.000
Epoch 0, Iteration 579, loss 0.342, batch accuracy 0.940
Epoch 0, Iteration 580, loss 0.040, batch accuracy 0.980
Epoch 0, Iteration 581, loss 0.096, batch accuracy 0.980
Epoch 0, Iteration 582, loss 0.041, batch accuracy 0.980
Epoch 0, Iteration 583, loss 0.022, batch accuracy 1.000
Epoch 0, Iteration 584, loss 0.117, batch accuracy 0.960
Epoch 0, Iteration 585, loss 0.066, batch accuracy 0.980
Epoch 0, Iteration 586, loss 0.032, batch accuracy 0.980
Epoch 0, Iteration 587, loss 0.024, batch accuracy 1.000
Epoch 0, Iteration 588, loss 0.036, batch accuracy 0.980
Epoch 0, Iteration 589, loss 0.099, batch accuracy 0.940
Epoch 0, Iteration 590, loss 0.126, batch accuracy 0.940
Epoch 0, Iteration 591, loss 0.022, batch accuracy 1.000
Epoch 0, Iteration 592, loss 0.127, batch accuracy 0.980
Epoch 0, Iteration 593, loss 0.177, batch accuracy 0.980
Epoch 0, Iteration 594, loss 0.042, batch accuracy 0.980
Epoch 0, Iteration 595, loss 0.

...

Epoch 1, Iteration 529, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 530, loss 0.037, batch accuracy 0.980
Epoch 1, Iteration 531, loss 0.041, batch accuracy 0.980
Epoch 1, Iteration 532, loss 0.126, batch accuracy 0.960
Epoch 1, Iteration 533, loss 0.050, batch accuracy 0.980
Epoch 1, Iteration 534, loss 0.048, batch accuracy 0.980
Epoch 1, Iteration 535, loss 0.037, batch accuracy 0.980
Epoch 1, Iteration 536, loss 0.058, batch accuracy 0.960
Epoch 1, Iteration 537, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 538, loss 0.149, batch accuracy 0.960
Epoch 1, Iteration 539, loss 0.025, batch accuracy 1.000
Epoch 1, Iteration 540, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 541, loss 0.114, batch accuracy 0.960
Epoch 1, Iteration 542, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 543, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 544, loss 0.000, batch accuracy 1.000
Epoch 1, Iteration 545, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 546, loss 0.

Epoch 1, Iteration 673, loss 0.024, batch accuracy 0.980
Epoch 1, Iteration 674, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 675, loss 0.053, batch accuracy 0.980
Epoch 1, Iteration 676, loss 0.046, batch accuracy 0.980
Epoch 1, Iteration 677, loss 0.043, batch accuracy 0.980
Epoch 1, Iteration 678, loss 0.008, batch accuracy 1.000
Epoch 1, Iteration 679, loss 0.029, batch accuracy 1.000
Epoch 1, Iteration 680, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 681, loss 0.025, batch accuracy 1.000
Epoch 1, Iteration 682, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 683, loss 0.060, batch accuracy 0.980
Epoch 1, Iteration 684, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 685, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 686, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 687, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 688, loss 0.043, batch accuracy 0.980
Epoch 1, Iteration 689, loss 0.104, batch accuracy 0.960
Epoch 1, Iteration 690, loss 0.

Epoch 1, Iteration 817, loss 0.164, batch accuracy 0.960
Epoch 1, Iteration 818, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 819, loss 0.042, batch accuracy 0.980
Epoch 1, Iteration 820, loss 0.085, batch accuracy 0.960
Epoch 1, Iteration 821, loss 0.005, batch accuracy 1.000
Epoch 1, Iteration 822, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 823, loss 0.152, batch accuracy 0.960
Epoch 1, Iteration 824, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 825, loss 0.115, batch accuracy 0.980
Epoch 1, Iteration 826, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 827, loss 0.029, batch accuracy 0.980
Epoch 1, Iteration 828, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 829, loss 0.008, batch accuracy 1.000
Epoch 1, Iteration 830, loss 0.108, batch accuracy 0.980
Epoch 1, Iteration 831, loss 0.057, batch accuracy 0.980
Epoch 1, Iteration 832, loss 0.044, batch accuracy 0.980
Epoch 1, Iteration 833, loss 0.021, batch accuracy 1.000
Epoch 1, Iteration 834, loss 0.

Epoch 1, Iteration 961, loss 0.013, batch accuracy 1.000
Epoch 1, Iteration 962, loss 0.014, batch accuracy 1.000
Epoch 1, Iteration 963, loss 0.110, batch accuracy 0.980
Epoch 1, Iteration 964, loss 0.058, batch accuracy 0.980
Epoch 1, Iteration 965, loss 0.037, batch accuracy 0.980
Epoch 1, Iteration 966, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 967, loss 0.019, batch accuracy 1.000
Epoch 1, Iteration 968, loss 0.023, batch accuracy 1.000
Epoch 1, Iteration 969, loss 0.032, batch accuracy 0.980
Epoch 1, Iteration 970, loss 0.017, batch accuracy 0.980
Epoch 1, Iteration 971, loss 0.069, batch accuracy 0.980
Epoch 1, Iteration 972, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 973, loss 0.144, batch accuracy 0.980
Epoch 1, Iteration 974, loss 0.067, batch accuracy 0.980
Epoch 1, Iteration 975, loss 0.024, batch accuracy 0.980
Epoch 1, Iteration 976, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 977, loss 0.029, batch accuracy 1.000
Epoch 1, Iteration 978, loss 0.

Epoch 1, Iteration 1103, loss 0.052, batch accuracy 0.980
Epoch 1, Iteration 1104, loss 0.059, batch accuracy 0.980
Epoch 1, Iteration 1105, loss 0.058, batch accuracy 0.960
Epoch 1, Iteration 1106, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 1107, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 1108, loss 0.055, batch accuracy 0.980
Epoch 1, Iteration 1109, loss 0.064, batch accuracy 0.980
Epoch 1, Iteration 1110, loss 0.050, batch accuracy 0.980
Epoch 1, Iteration 1111, loss 0.084, batch accuracy 0.960
Epoch 1, Iteration 1112, loss 0.021, batch accuracy 1.000
Epoch 1, Iteration 1113, loss 0.080, batch accuracy 0.960
Epoch 1, Iteration 1114, loss 0.047, batch accuracy 0.960
Epoch 1, Iteration 1115, loss 0.015, batch accuracy 1.000
Epoch 1, Iteration 1116, loss 0.005, batch accuracy 1.000
Epoch 1, Iteration 1117, loss 0.214, batch accuracy 0.940
Epoch 1, Iteration 1118, loss 0.032, batch accuracy 0.980
Epoch 1, Iteration 1119, loss 0.003, batch accuracy 1.000
Epoch 1, Itera

In [26]:
start_time = timeit.default_timer()

batch_size_test = 20
nb_test_points = x_test.shape[0] 
nb_iterations = nb_test_points//batch_size_test
preds = []
for i in range(nb_iterations):
    input_x_test = x_test[i*batch_size_test: (i+1)*batch_size_test]
    input_x_test = input_x_test[:, :, :,np.newaxis]
    preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
    preds.append(np.argmax(preds_test, axis=1))
    if np.mod(nb_test_points, batch_size_test) !=0:
        input_x_test = x_test[i*batch_size_test: -1]
        preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
        preds.append(np.argmax(preds, axis=1))
all_preds = np.concatenate(preds, axis =0)
y_real = np.argmax(y_test, axis=1)
acc_test = np.mean((all_preds==y_real)*1)
print('Test accuracy achieved: %.3f' %acc_test)

cnn_acc.append(acc_test)

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Test accuracy achieved: 0.988
Runtime:  59.89145661900011  seconds


#### CNN 5: 3 HL filters[32, 64, 128] k(2x2) s[1,1,1]

In [27]:
class CNN5:
    def __init__(self, x_train, y_train, output_dir, lr=0.001, nb_epochs=10, batch_size=50):
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.batch_size = batch_size
        self.nb_epochs = nb_epochs
        self.nb_images, self.edge, _ = x_train.shape
        self.nb_iterations = self.nb_images // batch_size
        self.output_dir = output_dir
        self.im = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.labels = tf.placeholder(tf.float32, [None, 10])
        self.x_train = x_train[:,:,:, np.newaxis]
        self.y_train = y_train
        
    def create_model(self):
        with tf.variable_scope('CNN5', reuse=tf.AUTO_REUSE):
             
            hl_1 = tf.layers.conv2d(self.im, 32, kernel_size=(2, 2), strides=1, activation=tf.nn.relu, name='hl_1')
            hl_2 = tf.layers.conv2d(hl_1, 64, kernel_size=(2, 2), strides=1, activation=tf.nn.relu, name='hl_2')
            hl_3 = tf.layers.conv2d(hl_2, 128, kernel_size=(2, 2), strides=1, activation=tf.nn.relu, name='hl_3')

            self.logits = tf.layers.dense(tf.layers.flatten(hl_3),10,activation=None, name='logits')
            self.preds = tf.nn.softmax(self.logits, name='preds')
            
    
    def compute_loss(self):
        with tf.variable_scope('loss'):
#             self.loss = tf.losses.softmax_cross_entropy(self.labels, self.logits)
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.labels,logits=self.logits))
            #########################################
            self.loss_summ = tf.summary.scalar("softmax_loss", self.loss)
    def optimizer(self):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5)
            self.model_vars = tf.trainable_variables()
            self.trainer = optimizer.minimize(self.loss, var_list=self.model_vars)

In [28]:
edge = int(np.sqrt(nb_features))
x_train.resize([n_train, edge, edge])
x_test.resize([n_test, edge, edge])

In [29]:
model = CNN5(x_train, y_train, './CNN5_logdir/', 0.001, 2, 50)
model.create_model()
model.compute_loss()
model.optimizer()
init = (tf.global_variables_initializer(),
        tf.local_variables_initializer())

saver = tf.train.Saver()
summary =tf.Summary()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.summary.FileWriter(model.output_dir)
writer.add_graph(sess.graph)
if not os.path.exists(model.output_dir):
    os.makedirs(model.output_dir)

In [30]:
start_time = timeit.default_timer()

for epoch in range(model.nb_epochs):
    randomize = np.arange(x_train.shape[0])
    np.random.shuffle(randomize)
    x_in = model.x_train[randomize,:]
    y_in = model.y_train[randomize,:]
    for i in range(model.nb_iterations):
        input_x_train = x_in[i*model.batch_size: (i+1)*model.batch_size]
        input_y_train = y_in[i*model.batch_size: (i+1)*model.batch_size]
        _ , preds, loss, loss_summ = sess.run([model.trainer, model.preds, model.loss, model.loss_summ], 
                                 feed_dict={model.im: input_x_train, 
                                            model.labels: input_y_train})
        y_preds = np.argmax(preds, axis=1)
        y_real = np.argmax(input_y_train, axis=1)
        acc_train = np.mean((y_preds==y_real)*1)
        print('Epoch %d, Iteration %d, loss %.3f, batch accuracy %.3f' %(epoch, i, loss, acc_train))
        writer.add_summary(loss_summ, epoch * model.nb_iterations + i)
    saver.save(sess, model.output_dir, global_step=epoch)  
# sess.close()

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Epoch 0, Iteration 0, loss 2.294, batch accuracy 0.160
Epoch 0, Iteration 1, loss 3.133, batch accuracy 0.240
Epoch 0, Iteration 2, loss 2.139, batch accuracy 0.260
Epoch 0, Iteration 3, loss 1.989, batch accuracy 0.380
Epoch 0, Iteration 4, loss 1.780, batch accuracy 0.340
Epoch 0, Iteration 5, loss 1.655, batch accuracy 0.560
Epoch 0, Iteration 6, loss 1.097, batch accuracy 0.800
Epoch 0, Iteration 7, loss 1.027, batch accuracy 0.660
Epoch 0, Iteration 8, loss 1.072, batch accuracy 0.680
Epoch 0, Iteration 9, loss 0.997, batch accuracy 0.740
Epoch 0, Iteration 10, loss 0.764, batch accuracy 0.740
Epoch 0, Iteration 11, loss 0.734, batch accuracy 0.740
Epoch 0, Iteration 12, loss 0.866, batch accuracy 0.720
Epoch 0, Iteration 13, loss 0.730, batch accuracy 0.620
Epoch 0, Iteration 14, loss 0.401, batch accuracy 0.840
Epoch 0, Iteration 15, loss 0.447, batch accuracy 0.860
Epoch 0, Iteration 16, loss 0.818, batch accuracy 0.780
Epoch 0, Iteration 17, loss 0.777, batch accuracy 0.740
Ep

Epoch 0, Iteration 146, loss 0.084, batch accuracy 0.960
Epoch 0, Iteration 147, loss 0.237, batch accuracy 0.920
Epoch 0, Iteration 148, loss 0.196, batch accuracy 0.940
Epoch 0, Iteration 149, loss 0.174, batch accuracy 0.940
Epoch 0, Iteration 150, loss 0.084, batch accuracy 0.980
Epoch 0, Iteration 151, loss 0.166, batch accuracy 0.980
Epoch 0, Iteration 152, loss 0.311, batch accuracy 0.940
Epoch 0, Iteration 153, loss 0.595, batch accuracy 0.920
Epoch 0, Iteration 154, loss 0.295, batch accuracy 0.920
Epoch 0, Iteration 155, loss 0.105, batch accuracy 0.980
Epoch 0, Iteration 156, loss 0.220, batch accuracy 0.960
Epoch 0, Iteration 157, loss 0.099, batch accuracy 0.980
Epoch 0, Iteration 158, loss 0.224, batch accuracy 0.940
Epoch 0, Iteration 159, loss 0.238, batch accuracy 0.920
Epoch 0, Iteration 160, loss 0.254, batch accuracy 0.940
Epoch 0, Iteration 161, loss 0.067, batch accuracy 0.980
Epoch 0, Iteration 162, loss 0.103, batch accuracy 0.980
Epoch 0, Iteration 163, loss 0.

Epoch 0, Iteration 290, loss 0.076, batch accuracy 0.960
Epoch 0, Iteration 291, loss 0.102, batch accuracy 0.980
Epoch 0, Iteration 292, loss 0.030, batch accuracy 0.980
Epoch 0, Iteration 293, loss 0.075, batch accuracy 0.960
Epoch 0, Iteration 294, loss 0.091, batch accuracy 0.980
Epoch 0, Iteration 295, loss 0.064, batch accuracy 0.980
Epoch 0, Iteration 296, loss 0.219, batch accuracy 0.940
Epoch 0, Iteration 297, loss 0.029, batch accuracy 0.980
Epoch 0, Iteration 298, loss 0.046, batch accuracy 0.980
Epoch 0, Iteration 299, loss 0.049, batch accuracy 0.980
Epoch 0, Iteration 300, loss 0.055, batch accuracy 0.980
Epoch 0, Iteration 301, loss 0.020, batch accuracy 1.000
Epoch 0, Iteration 302, loss 0.053, batch accuracy 0.980
Epoch 0, Iteration 303, loss 0.060, batch accuracy 0.980
Epoch 0, Iteration 304, loss 0.168, batch accuracy 0.940
Epoch 0, Iteration 305, loss 0.193, batch accuracy 0.960
Epoch 0, Iteration 306, loss 0.112, batch accuracy 0.960
Epoch 0, Iteration 307, loss 0.

Epoch 0, Iteration 434, loss 0.048, batch accuracy 0.980
Epoch 0, Iteration 435, loss 0.101, batch accuracy 0.980
Epoch 0, Iteration 436, loss 0.032, batch accuracy 1.000
Epoch 0, Iteration 437, loss 0.388, batch accuracy 0.920
Epoch 0, Iteration 438, loss 0.041, batch accuracy 0.980
Epoch 0, Iteration 439, loss 0.194, batch accuracy 0.960
Epoch 0, Iteration 440, loss 0.197, batch accuracy 0.940
Epoch 0, Iteration 441, loss 0.038, batch accuracy 0.980
Epoch 0, Iteration 442, loss 0.100, batch accuracy 0.940
Epoch 0, Iteration 443, loss 0.022, batch accuracy 1.000
Epoch 0, Iteration 444, loss 0.059, batch accuracy 0.960
Epoch 0, Iteration 445, loss 0.032, batch accuracy 1.000
Epoch 0, Iteration 446, loss 0.033, batch accuracy 1.000
Epoch 0, Iteration 447, loss 0.046, batch accuracy 1.000
Epoch 0, Iteration 448, loss 0.194, batch accuracy 0.980
Epoch 0, Iteration 449, loss 0.039, batch accuracy 0.980
Epoch 0, Iteration 450, loss 0.081, batch accuracy 0.980
Epoch 0, Iteration 451, loss 0.

Epoch 0, Iteration 578, loss 0.129, batch accuracy 0.980
Epoch 0, Iteration 579, loss 0.017, batch accuracy 1.000
Epoch 0, Iteration 580, loss 0.050, batch accuracy 1.000
Epoch 0, Iteration 581, loss 0.016, batch accuracy 1.000
Epoch 0, Iteration 582, loss 0.015, batch accuracy 1.000
Epoch 0, Iteration 583, loss 0.024, batch accuracy 1.000
Epoch 0, Iteration 584, loss 0.055, batch accuracy 0.980
Epoch 0, Iteration 585, loss 0.070, batch accuracy 0.980
Epoch 0, Iteration 586, loss 0.120, batch accuracy 0.960
Epoch 0, Iteration 587, loss 0.336, batch accuracy 0.960
Epoch 0, Iteration 588, loss 0.110, batch accuracy 0.940
Epoch 0, Iteration 589, loss 0.015, batch accuracy 1.000
Epoch 0, Iteration 590, loss 0.041, batch accuracy 1.000
Epoch 0, Iteration 591, loss 0.102, batch accuracy 0.980
Epoch 0, Iteration 592, loss 0.100, batch accuracy 0.960
Epoch 0, Iteration 593, loss 0.044, batch accuracy 1.000
Epoch 0, Iteration 594, loss 0.180, batch accuracy 0.960
Epoch 0, Iteration 595, loss 0.

...

Epoch 1, Iteration 529, loss 0.031, batch accuracy 1.000
Epoch 1, Iteration 530, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 531, loss 0.064, batch accuracy 0.980
Epoch 1, Iteration 532, loss 0.082, batch accuracy 0.960
Epoch 1, Iteration 533, loss 0.059, batch accuracy 0.980
Epoch 1, Iteration 534, loss 0.029, batch accuracy 1.000
Epoch 1, Iteration 535, loss 0.013, batch accuracy 1.000
Epoch 1, Iteration 536, loss 0.020, batch accuracy 0.980
Epoch 1, Iteration 537, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 538, loss 0.008, batch accuracy 1.000
Epoch 1, Iteration 539, loss 0.054, batch accuracy 0.980
Epoch 1, Iteration 540, loss 0.013, batch accuracy 1.000
Epoch 1, Iteration 541, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 542, loss 0.021, batch accuracy 0.980
Epoch 1, Iteration 543, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 544, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 545, loss 0.072, batch accuracy 0.980
Epoch 1, Iteration 546, loss 0.

Epoch 1, Iteration 673, loss 0.099, batch accuracy 0.980
Epoch 1, Iteration 674, loss 0.131, batch accuracy 0.960
Epoch 1, Iteration 675, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 676, loss 0.023, batch accuracy 1.000
Epoch 1, Iteration 677, loss 0.032, batch accuracy 0.980
Epoch 1, Iteration 678, loss 0.063, batch accuracy 0.980
Epoch 1, Iteration 679, loss 0.033, batch accuracy 1.000
Epoch 1, Iteration 680, loss 0.022, batch accuracy 0.980
Epoch 1, Iteration 681, loss 0.022, batch accuracy 1.000
Epoch 1, Iteration 682, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 683, loss 0.068, batch accuracy 0.960
Epoch 1, Iteration 684, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 685, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 686, loss 0.036, batch accuracy 0.980
Epoch 1, Iteration 687, loss 0.071, batch accuracy 0.960
Epoch 1, Iteration 688, loss 0.081, batch accuracy 0.960
Epoch 1, Iteration 689, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 690, loss 0.

Epoch 1, Iteration 817, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 818, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 819, loss 0.025, batch accuracy 1.000
Epoch 1, Iteration 820, loss 0.036, batch accuracy 1.000
Epoch 1, Iteration 821, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 822, loss 0.032, batch accuracy 1.000
Epoch 1, Iteration 823, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 824, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 825, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 826, loss 0.028, batch accuracy 1.000
Epoch 1, Iteration 827, loss 0.010, batch accuracy 1.000
Epoch 1, Iteration 828, loss 0.187, batch accuracy 0.960
Epoch 1, Iteration 829, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 830, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 831, loss 0.065, batch accuracy 0.960
Epoch 1, Iteration 832, loss 0.054, batch accuracy 0.980
Epoch 1, Iteration 833, loss 0.024, batch accuracy 1.000
Epoch 1, Iteration 834, loss 0.

Epoch 1, Iteration 961, loss 0.026, batch accuracy 1.000
Epoch 1, Iteration 962, loss 0.037, batch accuracy 0.980
Epoch 1, Iteration 963, loss 0.019, batch accuracy 1.000
Epoch 1, Iteration 964, loss 0.041, batch accuracy 0.980
Epoch 1, Iteration 965, loss 0.020, batch accuracy 1.000
Epoch 1, Iteration 966, loss 0.038, batch accuracy 0.980
Epoch 1, Iteration 967, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 968, loss 0.074, batch accuracy 0.980
Epoch 1, Iteration 969, loss 0.005, batch accuracy 1.000
Epoch 1, Iteration 970, loss 0.037, batch accuracy 0.980
Epoch 1, Iteration 971, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 972, loss 0.201, batch accuracy 0.980
Epoch 1, Iteration 973, loss 0.143, batch accuracy 0.980
Epoch 1, Iteration 974, loss 0.005, batch accuracy 1.000
Epoch 1, Iteration 975, loss 0.015, batch accuracy 1.000
Epoch 1, Iteration 976, loss 0.007, batch accuracy 1.000
Epoch 1, Iteration 977, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 978, loss 0.

Epoch 1, Iteration 1103, loss 0.019, batch accuracy 1.000
Epoch 1, Iteration 1104, loss 0.148, batch accuracy 0.960
Epoch 1, Iteration 1105, loss 0.011, batch accuracy 1.000
Epoch 1, Iteration 1106, loss 0.098, batch accuracy 0.960
Epoch 1, Iteration 1107, loss 0.022, batch accuracy 0.980
Epoch 1, Iteration 1108, loss 0.040, batch accuracy 1.000
Epoch 1, Iteration 1109, loss 0.031, batch accuracy 0.980
Epoch 1, Iteration 1110, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 1111, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 1112, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 1113, loss 0.063, batch accuracy 0.980
Epoch 1, Iteration 1114, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 1115, loss 0.181, batch accuracy 0.980
Epoch 1, Iteration 1116, loss 0.035, batch accuracy 0.980
Epoch 1, Iteration 1117, loss 0.024, batch accuracy 0.980
Epoch 1, Iteration 1118, loss 0.034, batch accuracy 0.980
Epoch 1, Iteration 1119, loss 0.135, batch accuracy 0.980
Epoch 1, Itera

In [31]:
start_time = timeit.default_timer()

batch_size_test = 20
nb_test_points = x_test.shape[0] 
nb_iterations = nb_test_points//batch_size_test
preds = []
for i in range(nb_iterations):
    input_x_test = x_test[i*batch_size_test: (i+1)*batch_size_test]
    input_x_test = input_x_test[:, :, :,np.newaxis]
    preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
    preds.append(np.argmax(preds_test, axis=1))
    if np.mod(nb_test_points, batch_size_test) !=0:
        input_x_test = x_test[i*batch_size_test: -1]
        preds_test = sess.run(model.preds, 
                             feed_dict={model.im: input_x_test})
        preds.append(np.argmax(preds, axis=1))
all_preds = np.concatenate(preds, axis =0)
y_real = np.argmax(y_test, axis=1)
acc_test = np.mean((all_preds==y_real)*1)
print('Test accuracy achieved: %.3f' %acc_test)

cnn_acc.append(acc_test)

elapsed = timeit.default_timer() - start_time      # Runtime counter stops
print('Runtime: ' , elapsed, ' seconds')

Test accuracy achieved: 0.984
Runtime:  33.402860328000315  seconds


In [2]:
x = [1, 2, 3, 4, 5]
y = [98.9, 98.6, 98.9, 98.5, 98.2]

plt.bar(x,y, color='k')
bottom = 97
top = 100
plt.ylim((bottom, top)) 
plt.xlabel('CNNs of varied Depths and Complexities')
plt.ylabel('Accuracy(%)')
plt.title('Graph for Accuracy vs Depth and Complexities')
plt.show()